In [1]:
import matplotlib.pyplot as pyplot
%matplotlib inline

In [2]:
# loading scripts are in scripts/dataset.py
from scripts import dataset
import chainer
import os

# example code from tests/test_snapshot.py
dataset_name = 'lsp'
#dataset_name = 'lsp'

if dataset_name == 'mpii':
    DATASET_ROOT = os.path.expanduser('~/data/mpii')  # MPII dataset is here
    TEST_CV_FILEPATH = os.path.join(DATASET_ROOT, 'test_joints.csv')
    IMG_PATH_PREFIX = os.path.join(MPII_DATASET_ROOT, 'images')
    symmetric_joints = "[[12, 13], [11, 14], [10, 15], [2, 3], [1, 4], [0, 5]]"
    ignore_label = -100500
else:
    DATASET_ROOT = os.path.expanduser('~/data/lsp')  # LSP dataset is here
    TEST_CV_FILEPATH = os.path.join(DATASET_ROOT, 'test_joints.csv')
    IMG_PATH_PREFIX = os.path.join(DATASET_ROOT, 'images')
    symmetric_joints = "[[8, 9], [7, 10], [6, 11], [2, 3], [1, 4], [0, 5]]"
    ignore_label = -1

test_dataset = dataset.PoseDataset(
        TEST_CV_FILEPATH,
        IMG_PATH_PREFIX, 227,
        fliplr=False, rotate=False,
        shift=None,
        bbox_extension_range=(1.0, 1.0),
        coord_normalize=True,
        gcn=True,
        fname_index=0,
        joint_index=1,
        symmetric_joints=symmetric_joints,
        ignore_label=ignore_label,
        should_return_bbox=True,
        should_downscale_images=True,
        downscale_height=400
)


184it [00:00, 1826.55it/s]

Reading dataset from /home/wonjinlee/data/lsp/test_joints.csv
Downscale images to the height 400px


1000it [00:00, 1785.94it/s]


[('/home/wonjinlee/data/lsp/images/im1001.jpg', array([[  42.96321929,  138.35880369],
       [  72.80248592,  114.75865645],
       [  54.35639382,   90.88724314],
       [  47.84600837,   90.61597708],
       [  44.31954959,  121.26904189],
       [  20.71940235,  147.31058368],
       [  60.5955132 ,   80.03660073],
       [  43.23448535,   66.74456378],
       [  52.18626534,   46.39960926],
       [  62.76564169,   66.74456378],
       [  38.0804302 ,   64.5744353 ],
       [  46.76094413,   42.0593523 ],
       [  53.27132958,   38.80415958],
       [  53.54259564,   20.9005996 ]])), ('/home/wonjinlee/data/lsp/images/im1002.jpg', array([[  40.7491173 ,  132.30774442],
       [  59.33184077,  117.50455793],
       [  84.21379255,  100.81160294],
       [ 107.20597583,  111.52029104],
       [  76.96968001,  144.90620102],
       [  68.4657218 ,  137.03216564],
       [  88.93821377,   83.48872512],
       [  76.65471859,   86.00841644],
       [  91.7728665 ,   64.27607882],
     

In [3]:
whos

Variable           Type           Data/Info
-------------------------------------------
DATASET_ROOT       str            /home/wonjinlee/data/lsp
IMG_PATH_PREFIX    str            /home/wonjinlee/data/lsp/images
TEST_CV_FILEPATH   str            /home/wonjinlee/data/lsp/test_joints.csv
chainer            module         <module 'chainer' from '/<...>ges/chainer/__init__.py'>
dataset            module         <module 'scripts.dataset'<...>pose/scripts/dataset.py'>
dataset_name       str            lsp
ignore_label       int            -1
os                 module         <module 'os' from '/usr/lib/python3.5/os.py'>
pyplot             module         <module 'matplotlib.pyplo<...>es/matplotlib/pyplot.py'>
symmetric_joints   str            [[8, 9], [7, 10], [6, 11]<...>, [2, 3], [1, 4], [0, 5]]
test_dataset       PoseDataset    <scripts.dataset.PoseData<...>object at 0x7f4158c2ca20>


In [4]:
from chainer import iterators
test_iterator = iterators.MultiprocessIterator(
        test_dataset, batch_size=128,
        repeat=False, shuffle=False,
        n_processes=1, n_prefetch=1)

In [18]:
##code from def test_net(test_dataset, test_iterator, dataset_name, snapshot_path):
from scripts import regressionnet

## set the checkpoint that you want to use
snapshot_path=os.path.expanduser('~/deeppose/out/lsp_alexnet_imagenet_small/checkpoint-820000')

if dataset_name not in ['lsp', 'mpii']:
    raise ValueError('Unknown dataset: {}'.format(dataset_name))

net, loss_op, pose_loss_op, train_op = regressionnet.create_regression_net( # in scripts.
        n_joints=16 if dataset_name == 'mpii' else 14,
        init_snapshot_path=snapshot_path,
        is_resume=True,
        net_type='Alexnet',
        optimizer_type='momentum',
        gpu_memory_fraction=0.32)  # Set how much GPU memory to reserve for the network
print(snapshot_path)

Alexnet::__create_architecture()
Initializing conv1 with random
Initializing conv2 with random
Initializing conv3 with random
Initializing conv4 with random
Initializing conv5 with random
Initializing fc6 with random
Initializing fc7 with random
Initializing fc8 with random
Initializing layer 99 with random
Restoring everything from snapshot and resuming
INFO:tensorflow:Restoring parameters from /home/wonjinlee/deeppose/out/lsp_alexnet_imagenet_small/checkpoint-820000


INFO:tensorflow:Restoring parameters from /home/wonjinlee/deeppose/out/lsp_alexnet_imagenet_small/checkpoint-820000


Creating optimizer momentum
Conv LR: Tensor("lr/conv_lr:0", shape=(), dtype=float32), FC LR: Tensor("lr/fc_lr:0", shape=(), dtype=float32)
INFO:tensorflow:Summary name grad_norms/conv1/weight:0 is illegal; using grad_norms/conv1/weight_0 instead.


INFO:tensorflow:Summary name grad_norms/conv1/weight:0 is illegal; using grad_norms/conv1/weight_0 instead.


INFO:tensorflow:Summary name grad_norms/conv1/bias:0 is illegal; using grad_norms/conv1/bias_0 instead.


...

INFO:tensorflow:Summary name grad_norms/fc_regression/weight:0 is illegal; using grad_norms/fc_regression/weight_0 instead.


INFO:tensorflow:Summary name grad_norms/fc_regression/weight:0 is illegal; using grad_norms/fc_regression/weight_0 instead.


INFO:tensorflow:Summary name grad_norms/fc_regression/bias:0 is illegal; using grad_norms/fc_regression/bias_0 instead.


INFO:tensorflow:Summary name grad_norms/fc_regression/bias:0 is illegal; using grad_norms/fc_regression/bias_0 instead.


uninit vars: ['conv1/weight/Momentum:0', 'conv1/bias/Momentum:0', 'conv2/weight/Momentum:0', 'conv2/bias/Momentum:0', 'conv3/weight/Momentum:0', 'conv3/bias/Momentum:0', 'conv4/weight/Momentum:0', 'conv4/bias/Momentum:0', 'conv5/weight/Momentum:0', 'conv5/bias/Momentum:0', 'fc6/weight/Momentum:0', 'fc6/bias/Momentum:0', 'fc7/weight/Momentum:0', 'fc7/bias/Momentum:0', 'fc_regression/weight/Momentum:0', 'fc_regression/bias/Momentum:0']
Elapsed time for finding uninitialized variables: 0.64s
Elapsed time to init them: 0.20s
/home/wonjinlee/deeppose/out/lsp_alexnet_imagenet_small/checkpoint-820000


In [19]:
import numpy as np

for ext in np.linspace(1.0, 2.0, 6, True):
    print('\n====================')
    print('BBOX EXTENSION:', ext)
    test_dataset.bbox_extension_range = (ext, ext)
    regressionnet.evaluate_pcp(net, pose_loss_op, test_iterator, None, # in scripts.
                               dataset_name=dataset_name,
                               tag_prefix='test')

  0%|          | 0/8 [00:00<?, ?it/s]


BBOX EXTENSION: 1.0
1000


100%|██████████| 8/8 [00:04<00:00,  1.72it/s]


Step 820000 test/pose_loss = 0.074
lsp


/home/wonjinlee/deeppose/poseevaluation/pcp.py:88: RuntimeWarning: divide by zero encountered in float_scalars
  gt_joints[i]['sticks'][stick_id, :2]) / gt_stick_len
/home/wonjinlee/deeppose/poseevaluation/pcp.py:90: RuntimeWarning: divide by zero encountered in float_scalars
  gt_joints[i]['sticks'][stick_id, 2:]) / gt_stick_len


...

Step 820000 test/pose_loss = 0.027
lsp
Step	 820000	 test/mPCP	 0.503
Step 820000 test/parts_PCP:
Head	Torso	U Arm	L Arm	U Leg	L Leg	mean
0.518	0.866	0.344	0.142	0.653	0.495	0.503
lsp
Step	 820000	 test/mRelaxedPCP	 0.594
Step 820000 test/parts_RelaxedPCP:
Head	Torso	U Arm	L Arm	U Leg	L Leg	mean
0.610	0.909	0.448	0.233	0.758	0.608	0.594
lsp
Step	 820000	 test/mPCKh	 0.419
Step	 820000	 test/mSymmetricPCKh	 0.442
Step 820000 test/parts_SymmetricPCKh:
Head	Neck	Shoulder	Elbow	Wrist	Hip	Knee	Ankle
0.549	0.655	0.500	0.338	0.203	0.608	0.433	0.246


  0%|          | 0/8 [00:00<?, ?it/s]


BBOX EXTENSION: 2.0
1000


100%|██████████| 8/8 [00:04<00:00,  1.97it/s]


Step 820000 test/pose_loss = 0.027
lsp
Step	 820000	 test/mPCP	 0.503
Step 820000 test/parts_PCP:
Head	Torso	U Arm	L Arm	U Leg	L Leg	mean
0.519	0.868	0.345	0.143	0.653	0.492	0.503
lsp
Step	 820000	 test/mRelaxedPCP	 0.595
Step 820000 test/parts_RelaxedPCP:
Head	Torso	U Arm	L Arm	U Leg	L Leg	mean
0.611	0.909	0.449	0.234	0.759	0.610	0.595
lsp
Step	 820000	 test/mPCKh	 0.419
Step	 820000	 test/mSymmetricPCKh	 0.442
Step 820000 test/parts_SymmetricPCKh:
Head	Neck	Shoulder	Elbow	Wrist	Hip	Knee	Ankle
0.546	0.657	0.502	0.337	0.203	0.609	0.433	0.249
